In [1]:
import logging
from pathlib import Path
from Pegasus.api import *
import glob

In [2]:
logging.basicConfig(level=logging.DEBUG)

In [3]:
seperate = Transformation(
                "seperate",
                site="local",
                pfn="/home/scitech/shared-data/wf-3/increment.py",
                is_stageable=True,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

count = Transformation(
                "count",
                site="local",
                pfn="/home/scitech/shared-data/wf-3/count.py",
                is_stageable=True,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

tar = Transformation(
                "tar",
                site="condorpool",
                pfn="/bin/tar",
                is_stageable=False,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

In [4]:
tc = TransformationCatalog().add_transformations(seperate, count, tar)

In [5]:
wf = Workflow("workflow3")

In [6]:
file_list = []
rc = ReplicaCatalog()

for f in glob.glob("nums*.txt"):
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path(".").resolve() / f)

fb1 = File("odd_nums.txt")
fb2 = File("even_nums.txt")
fc1 = File("odd_count.txt")
fc2 = File("even_count.txt")

job_seperate = Job(seperate)\
                    .add_inputs(*file_list)\
                    .add_outputs(fb1, fb2)

job_count_odd = Job(count)\
                .add_args(fb1, fc1)\
                .add_inputs(fb1)\
                .add_outputs(fc1)

job_count_even = Job(count)\
                .add_args(fb2, fc2)\
                .add_inputs(fb2)\
                .add_outputs(fc2)

fc3 = File("result.tar.gz")

job_tar = Job(tar)\
            .add_args(fc1, fc2)\
            .add_inputs(fc1, fc2)\
            .set_stdout(fc3)



In [7]:
wf.add_jobs(job_seperate, job_count_odd, job_count_even, job_tar)
wf.add_replica_catalog(rc)
wf.add_transformation_catalog(tc)

In [ ]:
try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)

Plan:
2020.07.08 23:14:57.295 UTC:    
2020.07.08 23:14:57.301 UTC:   ----------------------------------------------------------------------- 
2020.07.08 23:14:57.306 UTC:   File for submitting this DAG to HTCondor           : workflow3-0.dag.condor.sub 
2020.07.08 23:14:57.312 UTC:   Log of DAGMan debugging messages                 : workflow3-0.dag.dagman.out 
2020.07.08 23:14:57.317 UTC:   Log of HTCondor library output                     : workflow3-0.dag.lib.out 
2020.07.08 23:14:57.324 UTC:   Log of HTCondor library error messages             : workflow3-0.dag.lib.err 
2020.07.08 23:14:57.330 UTC:   Log of the life of condor_dagman itself          : workflow3-0.dag.dagman.log 
2020.07.08 23:14:57.337 UTC:    
2020.07.08 23:14:57.344 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with: 
2020.07.08 23:14:57.356 UTC:   ----------------------------------------------------------------------- 
2020.07.08 23:14:58.417 UTC:   Your database is compatible with P

[######--------------------------------------------]  11.1% ..Running (Completed: 2, Queued: 0, Running: 1, Failed: 0)